In [1]:
import requests
import numpy as np
import pandas as pd
import os

In [2]:
import tqdm

## Load SHAC

In [3]:
import sys

In [4]:
sys.path.insert(0, "..")

In [5]:
from src.process_SHAC import load_process_SHAC

In [6]:
import joblib

In [7]:
from tqdm import tqdm


from tqdm.notebook import tqdm

In [16]:
df = load_process_SHAC(replaceNA="all")

In [17]:
df

,id,Drug,Alcohol,Tobacco,SubstanceAgg,set,location,text
0,3407,True,False,False,True,train,uw,Social History: Work: Unemployed Lives with m...
1,3313,False,True,False,True,train,uw,SOCIAL HISTORY: Living situation: Comfortabl...
2,3045,True,False,True,True,train,uw,HABITS: Tobacco Use: Approximately 20 pack-ye...
3,4399,False,True,True,True,train,uw,Social History: Immigrated from [LOCATION] in ...
4,3637,False,False,True,True,train,uw,HABITS: Tobacco Use: Formerly smoked 1-2 ciga...
...,...,...,...,...,...,...,...,...
4400,2884,True,True,False,True,test,mimic,Social History: lives w/ mother in [**Name2 (N...
4401,2728,False,False,True,True,test,mimic,Social History: lives with wife (recent nursin...
4402,2854,False,True,True,True,test,mimic,"Social History: quit smoking several yrs ago, ..."
4403,2726,False,False,False,False,test,mimic,"Social History: Denies Tob, EtOH, or Illicit d..."


In [8]:
tqdm.pandas()

## Load Hate Speech

In [8]:
import pathlib

In [9]:
df_dynGen = pd.read_csv("/bime-munin/xiruod/data/hateSpeech_Bulla2023/Dynamically-Generated-Hate-Speech-Dataset/Dynamically Generated Hate Dataset v0.2.3.csv",)
df_dynGen['label_binary'] = df_dynGen['label'].map({"hate":1, "nothate":0})

In [10]:
df_dynGen["dfSource"] = "dynGen"

In [11]:
df_dynGen['label'].unique()

array(['hate', 'nothate'], dtype=object)

In [12]:
df_dynGen['label_binary'].unique()

array([1, 0])

In [13]:
ls_allFiles = pathlib.Path("/bime-munin/xiruod/data/hateSpeech_Bulla2023/hate-speech-dataset/all_files/").glob("*.txt")

ls_id = []
ls_text = []

for ifile in ls_allFiles:
    ls_id.append(ifile.name.split(".txt")[0])
    with open(ifile, "r") as f:
        ls_text.append(f.read())

df_wsf_raw = pd.DataFrame({"file_id":ls_id, "text":ls_text})

df_wsf_annotation = pd.read_csv("/bime-munin/xiruod/data/hateSpeech_Bulla2023/hate-speech-dataset/annotations_metadata.csv")

df_wsf = df_wsf_raw.merge(df_wsf_annotation, on="file_id", how="inner")

In [14]:
df_wsf.head()

,file_id,text,user_id,subforum_id,num_contexts,label
0,13480343_3,It can cause rage .,573459,1345,0,noHate
1,13341044_1,Obama has membership in Prince Hall ( for blac...,572139,1345,0,noHate
2,14036870_1,London is a dead zone as far as anyone White i...,572058,1387,0,hate
3,13861984_1,There was a small Nigerian man that lived in t...,574905,1391,0,noHate
4,13477847_2,New Square Meats allegedly sold nearly 2 milli...,573739,1345,0,noHate


In [15]:
df_wsf['label'].unique()

array(['noHate', 'hate', 'relation', 'idk/skip'], dtype=object)

In [16]:
df_wsf = df_wsf[df_wsf['label'].isin(['hate','noHate'])].reset_index(drop=True)

df_wsf['label_binary'] = df_wsf['label'].map({"hate":1,"noHate":0})

In [17]:
df_wsf['label_binary'].unique()

array([0, 1])

In [18]:
df_wsf["dfSource"] = "wsf"

In [19]:
tqdm.pandas()

# Llama v1

## Embeddings

In [3]:
# def generateEmbeddings(text):
    
#     header = {'Content-Type': 'application/json'}
#     url ="http://localhost:8088/?task=embeddings&prompt=" + text.replace(" ","%20")
#     req = requests.get(url,headers=header)
#     r = dict(req.json())
    
#     return r

def generateEmbeddings(text):

    myobj = {'model': 'instruct','task': 'embeddings','instruction':'', 'input': text}
   
    header = {'Content-Type': 'application/json'}
    url = "http://localhost:8088/" 

    x = requests.post(url, json = myobj, headers=header)
    r = dict(x.json())
    
    return r

In [4]:

def generateEmbeddingsAverage(text):
    
    r = generateEmbeddings(text)
    
    ret = np.mean(np.array(list(r.values())), axis=0)
    
    return ret

In [ ]:
tmp = generateEmbeddings("Hello")

# Store All Embeddings

In [ ]:
df['LLaMaEmbeddings'] = df.progress_apply(lambda x: 
                                         generateEmbeddings(x['text'].replace("&"," and ")), 
                                         axis=1)

In [ ]:
df.to_pickle("../output/LLaMaEmbeddings/df_SHAC.pkl")

In [ ]:
# with open("/edata/CohenLybarger/xiruo_Project/cognitive_distortions_Transfer_Learning_GitLab/data/LLaMaEmbeddings/df_avh.pkl", 'wb') as f:
#     joblib.dump(value=df_avh, filename=f)

# Store Averaged Embeddings

In [ ]:
# out_file = "../output/LLaMaEmbeddings/df_Average_SHAC.csv"
# for idx, row in df.iterrows():
#     if idx == 3:
#         break
        
#     row['embedding'] = generateEmbeddingsAverage(row['text'])
#     row.to_frame().T.to_csv(out_file, index=False, mode='a', header=not os.path.exists(out_file))
#     # df.iloc[[idx]].to_csv(out_file, index=False, mode='a', header=not os.path.exists(out_file))
#     # tmp = df.iloc[[idx]].copy()
#     # tmp['embedding'] = generateEmbeddingsAverage(df.iloc[idx]['text'])





In [12]:
df['LLaMaEmbeddings'] = df.progress_apply(lambda x: 
                                         generateEmbeddingsAverage(x['text'].replace("&"," and ")), 
                                         axis=1)

df.to_pickle("../output/LLaMaEmbeddings/df_Average_SHAC.pkl")

  0%|          | 0/4405 [00:00<?, ?it/s]

# Generations

In [2]:
r = requests.get(url, headers=header)
sresp = ""
for i in r.iter_lines():
    sresp = sresp + i.decode() #replace(‘\\xe2\\x80\\x99’,“\’“)

NameError: name 'url' is not defined

In [20]:
r

<Response [200]>

In [25]:
def complete(prompt, chars=32):
    header = {"Content-Type": "application/json"}
    url ="http://localhost:8088/?task=generate:"+str(chars)+"&prompt="+prompt.replace(" ","%20")
    r = requests.get(url,headers=header)
    sresp = ""
    for i in r.iter_lines():
        sresp = sresp + i.decode()#replace(‘\\xe2\\x80\\x99’,“\’“)
    return sresp

In [28]:
complete("what is LLaMa by Meta? ", chars=100)

' what is LLaMa by Meta? 24/11/2016 76466465LLaMa, a new initiative by Meta, is a web-based community aimed at bringing together stakeholders in the fields of Language Technologies and Machine Translation. The main objective of LLaMa will be to facilitate research and development across MT and Language Technologies. It aims to do so by connecting people'

# Llama 2

In [20]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "2,0"


In [21]:
import torch
from transformers import LlamaTokenizer, LlamaModel , LlamaConfig


In [22]:
class train_config:
    def __init__(self):
        self.quantization: bool = False

    

In [23]:
globalconfig = train_config()

In [ ]:
globalconfig.device = "cuda:0"

In [24]:
# globalconfig.output_file = "../output/LLaMaEmbeddings/df_Average_SHAC_llamav2_7b.pkl"
# globalconfig.model_id="/bime-munin/llama2_hf/llama-2-7b_hf/"
# globalconfig.quantization = False

# globalconfig.output_file = "../output/LLaMaEmbeddings/df_Average_SHAC_llamav2_13b.pkl"
# globalconfig.model_id="/bime-munin/llama2_hf/llama-2-13b_hf/"
# globalconfig.quantization = False

# globalconfig.output_file = "../output/LLaMaEmbeddings/df_Average_SHAC_llamav2_70b.pkl"
# globalconfig.model_id="/bime-munin/llama2_hf/llama-2-70b_hf/"
# globalconfig.quantization = True

### Test Permutation
# globalconfig.model_id="/bime-munin/llama2_hf/llama-2-7b_hf/"
# output_file_template = "../output/LLaMaEmbeddings/df_Average_Permute_{}_llamav2_7b.pkl"
# globalconfig.quantization = False

# globalconfig.model_id="/bime-munin/llama2_hf/llama-2-13b_hf/"
# output_file_template = "../output/LLaMaEmbeddings/df_Average_Permute_{}_llamav2_13b.pkl"
# globalconfig.quantization = False

globalconfig.model_id="/bime-munin/llama2_hf/llama-2-70b_hf/"
output_file_template = "../output/LLaMaEmbeddings/df_Average_Permute_{}_llamav2_70b.pkl"
globalconfig.quantization = True

In [25]:
globalconfig.max_seq_length=1024

In [26]:
!ls /bime-munin/llama2_hf/llama-2-70b_hf/

config.json			  pytorch_model-00010-of-00015.bin
generation_config.json		  pytorch_model-00011-of-00015.bin
pytorch_model-00001-of-00015.bin  pytorch_model-00012-of-00015.bin
pytorch_model-00002-of-00015.bin  pytorch_model-00013-of-00015.bin
pytorch_model-00003-of-00015.bin  pytorch_model-00014-of-00015.bin
pytorch_model-00004-of-00015.bin  pytorch_model-00015-of-00015.bin
pytorch_model-00005-of-00015.bin  pytorch_model.bin.index.json
pytorch_model-00006-of-00015.bin  special_tokens_map.json
pytorch_model-00007-of-00015.bin  tokenizer_config.json
pytorch_model-00008-of-00015.bin  tokenizer.json
pytorch_model-00009-of-00015.bin  tokenizer.model


In [27]:
tokenizer = LlamaTokenizer.from_pretrained("/bime-munin/llama2_hf/llama-2-7b_hf/")


In [ ]:
model = LlamaModel.from_pretrained(globalconfig.model_id, 
                                    load_in_8bit=globalconfig.quantization, 
                                    # device_map="cuda:0", 
                                   device_map="auto",
                                    torch_dtype=torch.float16 if globalconfig.quantization else torch.float32,
                                )



## No Permutation

In [19]:
def generateV2EmbeddingsAverage(text):
    
    model_input = tokenizer([text], return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():


        tmp = model(**model_input)
    
        ret = tmp['last_hidden_state'].squeeze().mean(axis=0).cpu().numpy()
        
    
    
    return ret

In [20]:
df['LLaMaEmbeddings'] = df.progress_apply(lambda x: 
                                         generateV2EmbeddingsAverage(x['text']), 
                                         axis=1)

  0%|          | 0/4405 [00:00<?, ?it/s]

In [21]:
globalconfig.output_file

'../output/LLaMaEmbeddings/df_Average_SHAC_llamav2_70b.pkl'

In [22]:
df.to_pickle("../output/LLaMaEmbeddings/df_Average_SHAC_llamav2_70b_8quant.pkl")

In [23]:
df.iloc[0]["LLaMaEmbeddings"]

array([-0.04727,  0.1318 ,  0.02307, ..., -0.4001 ,  2.096  ,  1.3545 ],
      dtype=float16)

In [24]:
df.iloc[0]["LLaMaEmbeddings"].shape

(8192,)

In [ ]:
df_dynGen['LLaMaEmbeddings'] = df_dynGen.progress_apply(lambda x: 
                                         generateV2EmbeddingsAverage(x['text']), 
                                         axis=1)

  0%|          | 0/41144 [00:00<?, ?it/s]

In [ ]:
# df_dynGen.to_pickle("../output/LLaMaEmbeddings/df_Average_HS_dynGen_llamav2_7b.pkl")
# df_dynGen.to_pickle("../output/LLaMaEmbeddings/df_Average_HS_dynGen_llamav2_13b.pkl")
df_dynGen.to_pickle("../output/LLaMaEmbeddings/df_Average_HS_dynGen_llamav2_70b_8quant.pkl")

In [ ]:
df_wsf['LLaMaEmbeddings'] = df_wsf.progress_apply(lambda x: 
                                         generateV2EmbeddingsAverage(x['text']), 
                                         axis=1)

In [ ]:
# df_wsf.to_pickle("../output/LLaMaEmbeddings/df_Average_HS_wsf_llamav2_7b.pkl")
# df_wsf.to_pickle("../output/LLaMaEmbeddings/df_Average_HS_wsf_llamav2_13b.pkl")
df_wsf.to_pickle("../output/LLaMaEmbeddings/df_Average_HS_wsf_llamav2_70b_8quant.pkl")

In [ ]:
df_wsf

## Simple Permutation (shift 1 per position, to the right)

In [31]:
def generateV2EmbeddingsPermuteAverage(text):
    
    model_input = tokenizer([text], return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        tmp = model(**model_input)
            
    tmp = tmp['last_hidden_state'].squeeze()
    
    ret_ts = []
    for i,row in enumerate(tmp):
        _ = torch.roll(row, i)
        ret_ts.append(_)

    ret = torch.cat(ret_ts).view(tmp.shape).mean(axis=0).cpu().numpy()
        
    return ret

In [37]:
# df_name = "wsf"
df_name = "dynGen"

In [38]:
if df_name == "wsf":
    df = df_wsf.copy()
    globalconfig.output_file = output_file_template.format("HS_" + df_name)
elif df_name == "dynGen":
    df = df_dynGen.copy()
    globalconfig.output_file = output_file_template.format("HS_" + df_name)
else:
    os.error("ERROR df name!")
    
 

In [39]:
globalconfig.output_file

'../output/LLaMaEmbeddings/df_Average_Permute_HS_dynGen_llamav2_13b.pkl'

In [40]:
   
df['LLaMaEmbeddings'] = df.progress_apply(lambda x: 
                                         generateV2EmbeddingsPermuteAverage(x['text']), 
                                         axis=1)

  0%|          | 0/41144 [00:00<?, ?it/s]

In [41]:
df.to_pickle(globalconfig.output_file)